In [74]:
from PIL import Image
import imagehash
import os
import collections
import pandas as pd

In [5]:
path = 'D:/msc/kaggle_data/whales/'
train_path = path + 'train/'
test_path = path+'test/'
validation_path = path+'validation/'

# Load stuff

In [58]:
dirs = sorted(os.listdir(train_path))

#First index of known whale
#first_kw_idx = len(os.listdir(train_path + dirs[0]))

start_index = 0
fn2ind = dict()
ind2fn = dict()
ind2dir = dict()
ind2phash = dict()

for dr in dirs:
    files = os.listdir(train_path + dr)
    fn2ind.update(dict(zip(files, range(start_index, start_index + len(files)))))
    ind2dir.update(dict(zip(range(start_index, start_index + len(files)), [dr]*len(files))))
    ind2phash.update(dict(zip(range(start_index, start_index + len(files)), [imagehash.phash(Image.open(train_path + dr+'/'+file)) for file in files])))
    start_index+=len(files)

val_dirs = sorted(os.listdir(validation_path))
val_start_index = len(fn2ind)
for dr in val_dirs:
    files = os.listdir(validation_path + dr)
    fn2ind.update(dict(zip(files, range(val_start_index, val_start_index + len(files)))))
    ind2dir.update(dict(zip(range(val_start_index, val_start_index + len(files)), [dr]*len(files))))
    ind2phash.update(dict(zip(range(val_start_index, val_start_index + len(files)), [imagehash.phash(Image.open(validation_path + dr+'/'+file)) for file in files])))
    val_start_index+=len(files)
ind2fn = {b : a for a,b in fn2ind.items()}
num_whales = len(ind2fn)

In [39]:
test_dirs = sorted(os.listdir(test_path))
test_start_index = 0
test_fn2ind = dict()
test_ind2fn = dict()
test_ind2dir = dict()
test_ind2phash = dict()
for dr in test_dirs:
    files = os.listdir(test_path + dr)
    test_fn2ind.update(dict(zip(files, range(test_start_index, test_start_index + len(files)))))
    test_ind2dir.update(dict(zip(range(test_start_index, test_start_index + len(files)), [dr]*len(files))))
    test_ind2phash.update(dict(zip(range(test_start_index, test_start_index + len(files)), [imagehash.phash(Image.open(test_path + dr+'/'+file)) for file in files])))
    test_start_index+=len(files)

test_ind2fn = {b : a for a,b in test_fn2ind.items()}

test_num_whales = len(test_ind2fn)

# Break tr/val into classes (don't need this...)

In [32]:
def equivalence_partition(iterable, relation):
    """Partitions a set of objects into equivalence classes

    Args:
        iterable: collection of objects to be partitioned
        relation: equivalence relation. I.e. relation(o1,o2) evaluates to True
            if and only if o1 and o2 are equivalent

    Returns: classes, partitions
        classes: A sequence of sets. Each one is an equivalence class
        partitions: A dictionary mapping objects to equivalence classes
    """
    classes = []
    partitions = {}
    for o in iterable:  # for each object
        # find the class it is in
        found = False
        for c in classes:
            if relation(next(iter(c)), o):  # is it equivalent to this class?
                c.add(o)
                partitions[o] = c
                found = True
                break
        if not found:  # it is in a new class
            classes.append(set([o]))
            partitions[o] = classes[-1]
    return classes, partitions

In [33]:
def rel_eq(i,j):
    return(ind2phash[i] == ind2phash[j])

In [34]:
equivalence_partition(range(num_whales+val_num_whales), rel_eq)

([{0, 6183},
  {1, 6789},
  {2},
  {3},
  {4},
  {5},
  {6},
  {7, 9209},
  {8},
  {9},
  {10, 6920},
  {11},
  {12},
  {13},
  {14},
  {15},
  {16},
  {17, 4277},
  {18},
  {19},
  {20},
  {21},
  {22},
  {23},
  {24},
  {25},
  {26},
  {27},
  {28, 1438},
  {29, 9716},
  {30},
  {31},
  {32},
  {33, 1894},
  {34},
  {35},
  {36},
  {37},
  {38, 3769},
  {39},
  {40},
  {41},
  {42},
  {43},
  {44},
  {45},
  {46},
  {47},
  {48},
  {49},
  {50, 2113},
  {51},
  {52},
  {53},
  {54},
  {55},
  {56, 5993},
  {57},
  {58, 4717},
  {59},
  {60},
  {61},
  {62},
  {63},
  {64, 3233},
  {65},
  {66},
  {67},
  {68, 9606},
  {69},
  {70},
  {71},
  {72},
  {73},
  {74},
  {75},
  {76},
  {77, 4202},
  {78},
  {79},
  {80, 7485},
  {81},
  {82},
  {83},
  {84},
  {85},
  {86},
  {87},
  {88},
  {89},
  {90},
  {91, 6747},
  {92},
  {93},
  {94},
  {95, 9783},
  {96, 3935},
  {97, 1856},
  {98},
  {99},
  {100},
  {101},
  {102},
  {103, 4248},
  {104},
  {105},
  {106, 8799},
  {107},
  {108

In [36]:
ind2dir[0]+'/'+ind2fn[0]

'new_whale/0031c258.jpg'

In [37]:
ind2dir[6183]+'/'+ind2fn[6183]

'w_aae82f8/f130cb7d.jpg'

# For each test image, find a close one in tr/val

In [78]:
test2names = collections.defaultdict(lambda:[])

In [79]:
for i in range(test_num_whales):
    h = test_ind2phash[i]
    for j in range(num_whales):
        if(abs(h - ind2phash[j])<=3):
            test2names[i].append(ind2dir[j])

In [80]:
df = pd.DataFrame.from_dict(test2names, orient='index')

In [94]:
df['names'] = df.apply(lambda x: set(filter(None.__ne__,{x[0], x[1], x[2]})), axis=1)

In [99]:
df['file'] = df.apply(lambda x: test_ind2fn[x.name], axis=1)

In [101]:
del df[0]
del df[1]
del df[2]

In [103]:
df.to_csv('duplicates.csv')

# Play area

In [28]:
phash2ind_0 = collections.defaultdict(lambda:[])

coinc_pairs = 0
for i in range(num_whales+val_num_whales-1):
    for j in range(i+1, num_whales+val_num_whales):
        if(ind2phash[i] == ind2phash[j]):
            coinc_pairs+=1
coinc_pairs

779

In [31]:
coinc_pairs = 0
for i in range(num_whales+val_num_whales-1):
    for j in range(i+1, num_whales+val_num_whales):
        if(abs(ind2phash[i] - ind2phash[j])<=3):
            coinc_pairs+=1
coinc_pairs

809

In [12]:
#for i in range(num_whales):
for i in range(10):
    print(imagehash.phash(Image.open(train_path+ind2dir[i] + "/" + ind2fn[i])))

b84e9128da2ddb33
e39ac5329acd1ac6
e19ca2718d7a5e07
b0854f72e09da91f
e7919fc63361c948
8cc324bbd641dbb8
eb9f84a1f81b8685
af58d535a03c4c8f
c39cac4e395b952a
a9399486cd3466eb


In [ ]:
imagehash.